In [1]:
import numpy as np

from inversion_ideas import DataMisfit, TikhonovZero, ConjugateGradient
from regressor import LinearRegressor

In [2]:
n_params = 10
rng = np.random.default_rng(seed=4242)
true_model = rng.uniform(size=10)
true_model

array([0.78225148, 0.67148671, 0.2373809 , 0.17946133, 0.34662367,
       0.15210999, 0.31142952, 0.23900652, 0.54355731, 0.91770851])

In [3]:
# Build the X array
n_data = 25
shape = (n_data, n_params)
X = rng.uniform(size=n_data * n_params).reshape(shape)

In [4]:
synthetic_data = X @ true_model
maxabs = np.max(np.abs(synthetic_data))
noise = rng.normal(scale=1e-2 * maxabs, size=synthetic_data.size)
synthetic_data += noise
synthetic_data

array([2.83840696, 2.18091081, 2.00623242, 2.08333039, 2.01694883,
       2.7826232 , 2.10564027, 1.27333506, 2.08859855, 1.94177648,
       1.88492037, 2.92394733, 2.17231952, 3.08009275, 1.61670886,
       1.77403753, 2.67305005, 1.91413882, 2.42117827, 2.13991628,
       2.0153805 , 2.71388471, 2.65944255, 2.44416121, 3.14217523])

In [5]:
simulation = LinearRegressor(X)

In [6]:
uncertainty = 1e-2 * maxabs * np.ones_like(synthetic_data)
data_misfit = DataMisfit(synthetic_data, uncertainty, simulation)
smallness = TikhonovZero(n_params)

In [7]:
phi = data_misfit + 1e-3 * smallness
phi

φ(m) + 0.00 φ(m)

In [8]:
# initial_model = np.zeros(n_params)
initial_model = np.ones(n_params)
initial_model

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

## Minimize manually with `scipy.sparse.linalg.cg`

In [9]:
from scipy.sparse.linalg import cg

In [10]:
grad = phi.gradient(initial_model)
hess = phi.hessian(initial_model)

In [11]:
model_step, code = cg(hess, -grad)
model_step, code

(array([-0.19170539, -0.34613831, -0.7533893 , -0.80431568, -0.67286042,
        -0.84611586, -0.67926356, -0.74445741, -0.47639439, -0.08477345]),
 0)

In [12]:
inverted_model = initial_model + model_step

In [13]:
print("Result:")
print(inverted_model)
print()
print("True model:")
print(true_model)

Result:
[0.80829461 0.65386169 0.2466107  0.19568432 0.32713958 0.15388414
 0.32073644 0.25554259 0.52360561 0.91522655]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]


## Minimize with SciPy's `minimize`

In [14]:
from scipy.optimize import minimize

In [15]:
result =  minimize(phi, initial_model)
result

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 0.002966282077477033
        x: [ 8.079e-01  6.539e-01  2.467e-01  1.955e-01  3.274e-01
             1.538e-01  3.207e-01  2.558e-01  5.235e-01  9.155e-01]
      nit: 29
      jac: [ 1.904e-06 -2.508e-06 -3.394e-06  3.782e-06 -1.607e-07
             2.483e-06  5.235e-06 -3.208e-06 -4.490e-06 -2.773e-06]
 hess_inv: [[ 5.638e+00 -2.713e+00 ...  1.756e+00 -8.879e-01]
            [-2.713e+00  1.026e+01 ... -1.750e+00 -5.143e-01]
            ...
            [ 1.756e+00 -1.750e+00 ...  4.038e+00 -2.598e+00]
            [-8.879e-01 -5.143e-01 ... -2.598e+00  6.402e+00]]
     nfev: 330
     njev: 30

In [16]:
# The minimize already gives you the minimum model
inverted_model = result.x

In [17]:
print("Result:")
print(inverted_model)
print()
print("True model:")
print(true_model)

Result:
[0.8078707  0.65390247 0.24667559 0.19549621 0.32738896 0.1537966
 0.32071056 0.25575429 0.52345213 0.91548875]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]


In [18]:
result =  minimize(phi, initial_model, jac=phi.gradient, method="CG")
result

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 0.002966282072430307
       x: [ 8.079e-01  6.539e-01  2.467e-01  1.955e-01  3.274e-01
            1.537e-01  3.207e-01  2.558e-01  5.235e-01  9.156e-01]
     nit: 19
     jac: [ 2.008e-06 -1.263e-06 -3.647e-06 -2.470e-06  6.859e-07
           -4.247e-06 -1.304e-06 -1.340e-07  5.048e-06  6.503e-06]
    nfev: 40
    njev: 40

In [19]:
# The minimize already gives you the minimum model
inverted_model = result.x

In [20]:
print("Result:")
print(result.x)
print()
print("True model:")
print(true_model)

Result:
[0.80785055 0.65388731 0.24667538 0.19546809 0.32739103 0.15374396
 0.32066661 0.25580322 0.52348512 0.9155614 ]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]


In [21]:
result =  minimize(phi, initial_model, jac=phi.gradient, hess=phi.hessian, method="Newton-CG")
result

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 0.002966281699805373
       x: [ 8.078e-01  6.539e-01  2.467e-01  1.955e-01  3.274e-01
            1.538e-01  3.207e-01  2.558e-01  5.235e-01  9.155e-01]
     nit: 9
     jac: [-2.186e-08 -1.115e-08  1.721e-08  3.206e-09 -1.867e-08
            7.141e-09  1.098e-08 -4.793e-09  4.405e-08 -2.718e-08]
    nfev: 9
    njev: 9
    nhev: 9

In [22]:
# The minimize already gives you the minimum model
inverted_model = result.x

In [23]:
print("Result:")
print(result.x)
print()
print("True model:")
print(true_model)

Result:
[0.80784607 0.65391906 0.24670909 0.19547325 0.32739262 0.15377275
 0.32066928 0.25578036 0.5234614  0.91551217]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]


## Use `Minimizer` class

In [24]:
minimizer = ConjugateGradient()
minimizer

In [25]:
inverted_model = minimizer(phi, initial_model)

In [26]:
print("Result:")
print(inverted_model)
print()
print("True model:")
print(true_model)

Result:
[0.80829461 0.65386169 0.2466107  0.19568432 0.32713958 0.15388414
 0.32073644 0.25554259 0.52360561 0.91522655]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]
